In [1]:
import warnings
warnings.filterwarnings('ignore')

from scripts import utils

import os
import openai
openai.api_key = utils.get_openai_api_key()

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [2]:
from pprint import pprint

In [3]:
from trulens_eval import Tru

tru = Tru(database_file="rag_triad.sqlite")

🦑 Tru initialized with db url sqlite:///rag_triad.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


In [4]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["pdfs/eBook-How-to-Build-a-Career-in-AI.pdf"]
).load_data()

In [5]:
from llama_index import Document

document = Document(text="\n\n".\
                    join([doc.text for doc in documents]))

In [6]:
from scripts.utils import build_sentence_window_index

from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

sentence_index = build_sentence_window_index(
    document,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="sentence_index"
)

In [7]:
from scripts.utils import get_sentence_window_query_engine

sentence_window_engine = \
get_sentence_window_query_engine(sentence_index)

In [8]:
output = sentence_window_engine.query(
    "How do you create your AI portfolio?")
pprint(output.response)

('To create your AI portfolio, you should focus on building a collection of '
 'projects that demonstrate your skill progression. This can be achieved by '
 'starting with simpler projects and gradually increasing the complexity as '
 'you gain more experience. Additionally, it is important to ensure that your '
 'portfolio showcases a variety of AI techniques and applications to highlight '
 'your versatility and breadth of knowledge. By following this approach, you '
 'can effectively showcase your skills and attract potential employers in the '
 'field of AI.')


### Feedack Functions

In [9]:
import nest_asyncio

nest_asyncio.apply()

In [10]:
from trulens_eval import OpenAI as fOpenAI

provider = fOpenAI()

### Answer Relevance

In [12]:
from trulens_eval import Feedback

# cot  - chain of thought
f_qa_relevance = Feedback(
    provider.relevance_with_cot_reasons,
    name="Answer Relevance"
).on_input().on_output()

# .on_input_output()

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .


### Context Relevance

In [18]:
from trulens_eval import TruLlama

context_selection = TruLlama.select_source_nodes().node.text

In [ ]:
# import numpy as np


# f_qs_relevance = (
#     Feedback(provider.qs_relevance,
#              name="Context Relevance")
#     .on_input()
#     .on(context_selection)
#     .aggregate(np.mean)
# )

In [19]:
import numpy as np

# qs - question statement

f_qs_relevance = (
    Feedback(provider.qs_relevance_with_cot_reasons,
             name="Context Relevance")
    .on_input()
    .on(context_selection)
    .aggregate(np.mean)
)

✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input statement will be set to __record__.app.query.rets.source_nodes[:].node.text .


### Grounddness

In [20]:
from trulens_eval.feedback import Groundedness

grounded = Groundedness(groundedness_provider=provider)

In [21]:
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons,
             name="Groundedness"
            )
    .on(context_selection)
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


### Evaluation of the RAG application

In [22]:
from trulens_eval import TruLlama
from trulens_eval import FeedbackMode

tru_recorder = TruLlama(
    sentence_window_engine,
    app_id="App_1",
    feedbacks=[
        f_qa_relevance,
        f_qs_relevance,
        f_groundedness
    ]
)

In [29]:
eval_questions = []
with open('text/eval_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        eval_questions.append(item)

In [30]:
eval_questions

['What are the keys to building a career in AI?',
 'How can teamwork contribute to success in AI?',
 'What is the importance of networking in AI?',
 'What are some good habits to develop for a successful career?',
 'How can altruism be beneficial in building a career?',
 'What is imposter syndrome and how does it relate to AI?',
 'Who are some accomplished individuals who have experienced imposter syndrome?',
 'What is the first step to becoming good at AI?',
 'What are some common challenges in AI?',
 'Is it normal to find parts of AI challenging?']

In [31]:
eval_questions.append("How can I be successful in AI?")

In [33]:
eval_questions.extend(["How can pineapple be grown in nigeria?"])

In [34]:
pprint(eval_questions)

['What are the keys to building a career in AI?',
 'How can teamwork contribute to success in AI?',
 'What is the importance of networking in AI?',
 'What are some good habits to develop for a successful career?',
 'How can altruism be beneficial in building a career?',
 'What is imposter syndrome and how does it relate to AI?',
 'Who are some accomplished individuals who have experienced imposter '
 'syndrome?',
 'What is the first step to becoming good at AI?',
 'What are some common challenges in AI?',
 'Is it normal to find parts of AI challenging?',
 'How can I be successful in AI?',
 'How can pineapple be grown in nigeria?',
 'How can pineapple be grown in nigeria?']


In [35]:
for question in eval_questions:
    with tru_recorder as recording:
        sentence_window_engine.query(question)

In [36]:
records, feedback = tru.get_records_and_feedback(app_ids=[])
records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Answer Relevance,Groundedness,Context Relevance,Answer Relevance_calls,Groundedness_calls,Context Relevance_calls,latency,total_tokens,total_cost
0,App_1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_7c95279874030eca61d8063557cede4b,"""What are the keys to building a career in AI?""","""The keys to building a career in AI are learn...",-,"{""record_id"": ""record_hash_7c95279874030eca61d...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-01-15T15:36:08.736752"", ""...",2024-01-15T15:36:19.879437,0.8,1.000,0.45,[{'args': {'prompt': 'What are the keys to bui...,[{'args': {'source': 'Chapter 7: A Simple Fram...,[{'args': {'question': 'What are the keys to b...,11,514,0.000788
1,App_1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_b142ecf1ad1dd3e0794c696605302bc3,"""How can teamwork contribute to success in AI?""","""Teamwork can contribute to success in AI by a...",-,"{""record_id"": ""record_hash_b142ecf1ad1dd3e0794...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-01-15T15:36:20.682185"", ""...",2024-01-15T15:36:36.192520,0.9,0.950,0.65,[{'args': {'prompt': 'How can teamwork contrib...,[{'args': {'source': 'To get a project starte...,[{'args': {'question': 'How can teamwork contr...,15,662,0.001045
2,App_1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_bbb89b6a6897c74556cacbe67f648605,"""What is the importance of networking in AI?""","""Networking is important in AI because it allo...",-,"{""record_id"": ""record_hash_bbb89b6a6897c74556c...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-01-15T15:36:36.914921"", ""...",2024-01-15T15:36:58.681886,0.9,0.375,0.20,[{'args': {'prompt': 'What is the importance o...,[{'args': {'source': 'What is the hiring proce...,[{'args': {'question': 'What is the importance...,21,545,0.000873
3,App_1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_cb23d28347f57af8982b62f43b4f1a98,"""What are some good habits to develop for a su...","""Developing good habits in eating, exercise, s...",-,"{""record_id"": ""record_hash_cb23d28347f57af8982...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-01-15T15:36:59.613699"", ""...",2024-01-15T15:37:12.740227,0.9,0.500,0.50,[{'args': {'prompt': 'What are some good habit...,"[{'args': {'source': '(For example, many arti...",[{'args': {'question': 'What are some good hab...,13,447,0.000696
4,App_1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_e5efcb4992b0407eb4b238eb7599f59e,"""How can altruism be beneficial in building a ...","""Altruism can be beneficial in building a care...",-,"{""record_id"": ""record_hash_e5efcb4992b0407eb4b...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-01-15T15:37:13.545612"", ""...",2024-01-15T15:37:26.860747,0.9,0.500,0.45,[{'args': {'prompt': 'How can altruism be bene...,[{'args': {'source': 'Many successful people d...,[{'args': {'question': 'How can altruism be be...,13,547,0.000864


In [37]:
feedback

['Groundedness', 'Context Relevance', 'Answer Relevance']

In [38]:
import pandas as pd

pd.set_option("display.max_colwidth", None)
records[["input", "output"] + feedback]

,input,output,Groundedness,Context Relevance,Answer Relevance
0,"""What are the keys to building a career in AI?""","""The keys to building a career in AI are learning foundational technical skills, working on projects, and finding a job, all of which is supported by being part of a community.""",1.000000,0.45,0.8
1,"""How can teamwork contribute to success in AI?""","""Teamwork can contribute to success in AI by allowing individuals to leverage the expertise and insights of their colleagues. When working on larger AI projects that require collaboration, the ability to lead and work effectively with a team becomes crucial. By working together, team members can share their deep technical insights, make informed decisions about technical architecture or data collection, and ultimately improve the project. Additionally, being surrounded by colleagues who are dedicated, hardworking, and continuously learning can inspire individuals to do the same, leading to greater success in AI endeavors.""",0.950000,0.65,0.9
2,"""What is the importance of networking in AI?""","""Networking is important in AI because it allows individuals to connect with others who have experience and knowledge in the field. By reaching out to people in their network, such as friends or alumni, individuals can receive guidance and support as they navigate their career in AI. Additionally, attending meetups and engaging in informational interviews can help individuals expand their network and learn from professionals who hold positions aligned with their interests. This networking can provide valuable insights, help individuals stay updated on the evolving field of AI, and potentially open up opportunities for collaboration or job prospects in the future.""",0.375000,0.20,0.9
3,"""What are some good habits to develop for a successful career?""","""Developing good habits in eating, exercise, sleep, personal relationships, work, learning, and self-care can be beneficial for a successful career. These habits can help individuals move forward while staying healthy and contribute to their overall success in their professional lives.""",0.500000,0.50,0.9
4,"""How can altruism be beneficial in building a career?""","""Altruism can be beneficial in building a career by helping others even as one focuses on their own career growth. By aiming to lift others during every step of their own journey, individuals can achieve better outcomes for themselves. This can create a positive reputation and network, which can lead to new opportunities and collaborations. Additionally, helping others can provide a sense of fulfillment and purpose, which can contribute to overall career satisfaction and well-being.""",0.500000,0.45,0.9
5,"""What is imposter syndrome and how does it relate to AI?""","""Imposter syndrome is a psychological phenomenon where individuals doubt their own abilities and fear being exposed as a fraud, despite evidence of their competence. In the context of AI, newcomers to the field sometimes experience imposter syndrome, questioning whether they truly belong in the AI community and if they are capable of succeeding in the field. It is important to address imposter syndrome in order to encourage individuals to continue growing in AI and not let self-doubt hinder their progress.""",1.000000,0.85,1.0
6,"""Who are some accomplished individuals who have experienced imposter syndrome?""","""Sheryl Sandberg, Michelle Obama, Tom Hanks, and Mike Cannon-Brookes are some accomplished individuals who have experienced imposter syndrome.""",1.000000,0.65,0.8
7,"""What is the first step to becoming good at AI?""","""The first step to becoming good at AI is to learn foundational technical skills.""",1.000000,0.55,0.9
8,"""What are some common challenges in AI?""","""Some common challenges in AI include keeping up-to-date with evolving technology, finding suitable projects and estimating timelines and return on investment, managing the highly iterative nature of AI projects, colla

In [39]:
tru.get_leaderboard(app_ids=[])

,Groundedness,Context Relevance,Answer Relevance,latency,total_cost
app_id,,,,,
App_1,0.787238,0.465385,0.907692,15.076923,0.001007


In [45]:
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.1.74:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>